In [23]:
from google.colab import drive
path = '/content/drive/'
drive.mount(path)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [24]:
%cd /content/drive/MyDrive/Custom_Image_Classification/

/content/drive/MyDrive/Custom_Image_Classification


In [25]:
!ls

car_6.jpeg	      model.weights.h5	  simple_images  x_train.dat
model_structure.json  motorcycle_49.jpeg  van_18.jpeg	 y_train.dat


In [26]:
# Extract Features from the dataset

from pathlib import Path
import numpy as np
import joblib
from tensorflow import keras
from tensorflow.keras.preprocessing import image
from keras.applications import densenet

classes = ['car','van','train','bicycle','motorcycle']

paths = []

for c in classes:
    paths.append(Path("simple_images") / c)
    print(paths)

images = []
labels = []

class_index = 0
for path in paths:
    count_image = 0
    for img in path.glob("*.jpeg"):
        img = image.load_img(img,target_size=(224, 224))
        image_array = image.img_to_array(img)
        images.append(image_array)
        labels.append(class_index)
        count_image += 1
    print('The number of ', classes[class_index], count_image)
    class_index += 1

x_train = np.array(images)
y_train = np.array(labels)

# Normalize image data to 0-to-1 range
x_train = densenet.preprocess_input(x_train)

pretrained_nn = densenet.DenseNet169(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
features_x = pretrained_nn.predict(x_train)
joblib.dump(features_x, "x_train.dat")
joblib.dump(y_train, "y_train.dat")

[PosixPath('simple_images/car')]
[PosixPath('simple_images/car'), PosixPath('simple_images/van')]
[PosixPath('simple_images/car'), PosixPath('simple_images/van'), PosixPath('simple_images/train')]
[PosixPath('simple_images/car'), PosixPath('simple_images/van'), PosixPath('simple_images/train'), PosixPath('simple_images/bicycle')]
[PosixPath('simple_images/car'), PosixPath('simple_images/van'), PosixPath('simple_images/train'), PosixPath('simple_images/bicycle'), PosixPath('simple_images/motorcycle')]
The number of  car 80
The number of  van 79
The number of  train 92
The number of  bicycle 83
The number of  motorcycle 85
51877672/51877672 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
14/14 ━━━━━━━━━━━━━━━━━━━━ 46s 2s/step


['y_train.dat']

In [27]:
# Training a model

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from pathlib import Path
import joblib

# Load data set
x_train = joblib.load("x_train.dat")
y_train = joblib.load("y_train.dat")

# Create a model and add layers
model = Sequential()

model.add(Flatten(input_shape=x_train.shape[1:]))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))  # Output Layers # Numbers can change here

# Compile the model
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer="adam",
    metrics=['accuracy']
)

# Train the model
model.fit(
    x_train,
    y_train,
    epochs=100,
    shuffle=True
)

# Save neural network structure
model_structure = model.to_json()
f = Path("model_structure.json")
f.write_text(model_structure)

# Save neural network's trained weights
model.save_weights("model.weights.h5")


Epoch 1/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 3s 73ms/step - accuracy: 0.5042 - loss: 16.3877
Epoch 2/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9283 - loss: 1.1711
Epoch 3/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9464 - loss: 1.5920
Epoch 4/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9841 - loss: 0.2068
Epoch 5/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9764 - loss: 0.5026
Epoch 6/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9878 - loss: 0.2751
Epoch 7/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9924 - loss: 0.0331
Epoch 8/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9947 - loss: 0.0548
Epoch 9/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9983 - loss: 0.0057
Epoch 10/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9960 - loss: 0.0258
Epoch 11/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9931 - loss: 0.0923
Epoch 12/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste

In [28]:
# Predicting the image using the trained model

from keras.models import model_from_json
from pathlib import Path
from tensorflow.keras.preprocessing import image
import numpy as np
from keras.applications import densenet

# Load the json file that contains the model's structure
f = Path("model_structure.json")
model_structure = f.read_text()

# Recreate the Keras model object from the json data
model = model_from_json(model_structure)

# Re-load the model's trained weights
model.load_weights("model.weights.h5")

# Load an image file to test, resizing it to 224x224 pixels (as required by this model)
img = image.load_img("van_18.jpeg", target_size=(224, 224))

# Convert the image to a numpy array
image_array = image.img_to_array(img)

# Add a forth dimension to the image (since Keras expects a bunch of images, not a single image)
images = np.expand_dims(image_array, axis=0)

# Normalize the data
images = densenet.preprocess_input(images)

# Use the pre-trained neural network to extract features from our test image (the same way we did to train the model)
feature_extraction_model = densenet.DenseNet169(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
features = feature_extraction_model.predict(images)

# Given the extracted features, make a final prediction using our own model
results = model.predict(features)
print('Probability:', results)

predicted_class = np.argmax(results)
predicted_name = 'None'
if predicted_class==0:
    predicted_name = 'Car'
elif predicted_class==1:
    predicted_name = 'Van'
elif predicted_class==2:
    predicted_name = 'Train'
elif predicted_class==3:
    predicted_name = 'Bicycle'
else:
    predicted_name = 'Motorcycle'
print('This is',predicted_name,'with confidence:',results[0][np.argmax(results)]*100)

1/1 ━━━━━━━━━━━━━━━━━━━━ 19s 19s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 420ms/step
Probability: [[0. 1. 0. 0. 0.]]
This is Van with confidence: 100.0
